integrated test 01
--

Perform a simple raytrace using all (most) defined optics. 

This integrated test will verfiy that all optics can be run without errors.

Where possible analytical optics will be checked against mesh optics. For direct ray-to-ray comparsion check_bragg needs to be set to False.


In [ ]:
%matplotlib widget
import numpy as np
import xicsrt

from xicsrt.util import mirplot
from xicsrt.util import profiler

In [ ]:
def get_config():
    # 1.
    config = dict()

    # 2.
    config['general'] = {}
    config['general']['number_of_iter'] = 1
    config['general']['save_images'] = False
    config['general']['random_seed'] = 0
    config['general']['strict_config_check'] = False

    # 3.
    config['sources'] = {}
    config['sources']['source'] = {}
    config['sources']['source']['class_name'] = 'XicsrtSourceDirected'
    config['sources']['source']['intensity'] = 1e4
    config['sources']['source']['wavelength'] = 3.9492
    config['sources']['source']['spread'] = np.radians(10.0)
    config['sources']['source']['xsize'] = 0.00
    config['sources']['source']['ysize'] = 0.00
    config['sources']['source']['zsize'] = 0.00

    # 4.
    config['optics'] = {}
    config['optics']['crystal'] = {}
    config['optics']['crystal']['class_name'] = 'XicsrtOpticPlanarMirror'
    config['optics']['crystal']['check_size'] = True
    config['optics']['crystal']['origin'] = [0.0, 0.0,         0.80374151]
    config['optics']['crystal']['zaxis']  = [0.0, 0.59497864, -0.80374151]
    config['optics']['crystal']['xsize']  = 0.2
    config['optics']['crystal']['ysize']  = 0.2
    config['optics']['crystal']['radius'] = 1.0
    config['optics']['crystal']['radius_major'] = 1.0
    config['optics']['crystal']['radius_minor'] = 0.2 
    config['optics']['crystal']['mesh_size'] = (41,41)
    
    # Rocking curve FWHM in radians.
    # This is taken from x0h for quartz 1,1,-2,0
    # Darwin Curve, sigma: 48.070 urad
    # Darwin Curve, pi:    14.043 urad
    config['optics']['crystal']['crystal_spacing'] = 2.45676
    config['optics']['crystal']['rocking_type'] = 'gaussian'
    config['optics']['crystal']['rocking_fwhm'] = 48.070e-6
    config['optics']['crystal']['check_bragg'] = False


    # 5.
    config['optics']['detector'] = {}
    config['optics']['detector']['class_name'] = 'XicsrtOpticDetector'
    config['optics']['detector']['origin'] = [0.0,  0.76871290, 0.56904832]
    config['optics']['detector']['zaxis']  = [0.0, -0.95641806, 0.29200084]
    config['optics']['detector']['xsize']  = 0.4
    config['optics']['detector']['ysize']  = 0.2
    
    return config

def run(config):
    # 6.
    profiler.resetProfiler()
    profiler.startProfiler()

    results = xicsrt.raytrace(config)

    profiler.stopProfiler()
    profiler.report()
    
    return results

results_dict = {}

In [ ]:
config = get_config()
config['optics']['crystal']['class_name'] = 'XicsrtOpticPlanarMirror'
results = run(config)
results_dict[config['optics']['crystal']['class_name']] = results

In [ ]:
config = get_config()
config['optics']['crystal']['class_name'] = 'XicsrtOpticSphericalMirror'

results = run(config)
results_dict[config['optics']['crystal']['class_name']] = results

In [ ]:
config = get_config()
config['optics']['crystal']['class_name'] = 'XicsrtOpticPlanarCrystal'

results = run(config)
results_dict[config['optics']['crystal']['class_name']] = results

In [ ]:
config = get_config()
config['optics']['crystal']['class_name'] = 'XicsrtOpticSphericalCrystal'

results = run(config)
results_dict[config['optics']['crystal']['class_name']] = results

In [ ]:
config = get_config()
config['optics']['crystal']['class_name'] = 'XicsrtOpticCylindricalCrystal'

results = run(config)
results_dict[config['optics']['crystal']['class_name']] = results

In [ ]:
config = get_config()
config['optics']['crystal']['class_name'] = 'XicsrtOpticToroidalCrystal'

results_dict[config['optics']['crystal']['class_name']] = run(config)


In [ ]:
config = get_config()
config['optics']['crystal']['class_name'] = 'XicsrtOpticMeshSphericalCrystal'

results = run(config)
results_dict[config['optics']['crystal']['class_name']] = results

In [ ]:
config = get_config()
config['optics']['crystal']['class_name'] = 'XicsrtOpticMeshCylindricalCrystal'
config['optics']['crystal']['mesh_size'] = (41,41)

results = run(config)
results_dict[config['optics']['crystal']['class_name']] = results

In [ ]:
config = get_config()
config['optics']['crystal']['class_name'] = 'XicsrtOpticMeshToroidalCrystal'
config['optics']['crystal']['mesh_size'] = (41,41)

results = run(config)
results_dict[config['optics']['crystal']['class_name']] = results

In [ ]:
config = get_config()
config['optics']['crystal']['class_name'] = 'XicsrtOpticPlanarMosaicCrystal'

results = run(config)
results_dict[config['optics']['crystal']['class_name']] = results

In [ ]:
config = get_config()
config['optics']['crystal']['class_name'] = 'XicsrtOpticSphericalMosaicCrystal'

results = run(config)
results_dict[config['optics']['crystal']['class_name']] = results

In [ ]:
results = results_dict['XicsrtOpticCylindricalCrystal']

import xicsrt.visual.xicsrt_2d__matplotlib as xicsrt_2d
fig = xicsrt_2d.plot_intersect(results, 'detector', aspect='equal')

In [ ]:
results = results_dict['XicsrtOpticMeshCylindricalCrystal']

import xicsrt.visual.xicsrt_2d__matplotlib as xicsrt_2d
fig = xicsrt_2d.plot_intersect(results, 'detector', aspect='equal')

In [ ]:
def get_plot_dict(results, **kwargs):
    optic = 'detector'
    # Retrieve an object for the given optic/source.
    obj = xicsrt.get_element(results['config'], optic)

    # Transform from global coordinate to local optic coordinates
    origin_ext = results['found']['history'][optic]['origin']
    origin_loc = obj.point_to_local(origin_ext)

    # Use the mirplot utility to generate the plot.
    plotdict = {
        'type':'scatter',
        'x':origin_loc[:, 0],
        'y':origin_loc[:, 1],
        }
    for key in kwargs:
        plotdict[key] = kwargs[key]
        
    return plotdict

def plot_comparison(names, results_dict):
    plotlist = []
    for name in names:
        results = results_dict[name]
        plotlist.append(get_plot_dict(
            results,
            label=name,
            legend=True,
            ))

    fig = mirplot.plot_to_screen(plotlist)
    return fig

In [ ]:
# Compare Similar Shapes
names = [
    'XicsrtOpticSphericalCrystal',
    'XicsrtOpticMeshSphericalCrystal',
    ]
    
fig = plot_comparison(names, results_dict)

In [ ]:
# Compare Similar Shapes
names = [
    'XicsrtOpticCylindricalCrystal',
    'XicsrtOpticMeshCylindricalCrystal',
    ]
    
fig = plot_comparison(names, results_dict)

In [ ]:
# Compare Similar Shapes
names = [
    'XicsrtOpticToroidalCrystal',
    'XicsrtOpticMeshToroidalCrystal',
    ]
    
fig = plot_comparison(names, results_dict)

In [ ]:
results = results_dict['XicsrtOpticMeshToroidalCrystal']

import xicsrt.visual.xicsrt_3d__plotly as xicsrt_3d
fig = xicsrt_3d.figure()
xicsrt_3d.add_rays(results)
xicsrt_3d.add_optics(results['config'])
xicsrt_3d.add_sources(results['config'])
xicsrt_3d.show()